# HR Analytics  
https://datahack.analyticsvidhya.com/contest/wns-analytics-hackathon-2018-1/  
**Aleksey Shipitsyn**    
**2019-08-05**

## Data readigng and exploration

In [345]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [346]:
# data files names
folder = './Documents/Competitions/HR Analytics/'
file_train = 'train_LZdllcl.xls'
file_test = 'test_2umaH9m.xls'
file_submission = 'sample_submission_M0L0uXE.xls'


In [347]:
# read and check train data
df_train = pd.read_csv(folder + file_train) 

print('DF shape: {}\n'.format(df_train.shape))
print('TYPES:\n\n{}\n\n'.format(df_train.dtypes))
print('MISSING VALUES:\n\n{}'.format(df_train.isnull().sum()))
df_train.head()



DF shape: (54808, 14)

TYPES:

employee_id               int64
department               object
region                   object
education                object
gender                   object
recruitment_channel      object
no_of_trainings           int64
age                       int64
previous_year_rating    float64
length_of_service         int64
KPIs_met >80%             int64
awards_won?               int64
avg_training_score        int64
is_promoted               int64
dtype: object


MISSING VALUES:

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64


,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [348]:
# balance of target check
pd.crosstab(index=df_train.is_promoted, columns=['is_promoted'], normalize=True).round(2)



col_0,is_promoted
is_promoted,
0,0.91
1,0.09


In [349]:
# read and check test data
df_test = pd.read_csv(folder + file_test) 

print('DF shape: {}\n'.format(df_test.shape))
print('TYPES:\n\n{}\n\n'.format(df_test.dtypes))
print('MISSING VALUES:\n\n{}'.format(df_test.isnull().sum()))
df_test.head()


DF shape: (23490, 13)

TYPES:

employee_id               int64
department               object
region                   object
education                object
gender                   object
recruitment_channel      object
no_of_trainings           int64
age                       int64
previous_year_rating    float64
length_of_service         int64
KPIs_met >80%             int64
awards_won?               int64
avg_training_score        int64
dtype: object


MISSING VALUES:

employee_id                0
department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64


,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [350]:
# read submission data
df_submission = pd.read_csv(folder + file_submission)

# check if submission hash corresponds to test set
print('ID match with test set: {}\n'.format(all(df_test.employee_id == df_submission.employee_id)))

print('DF shape: {}\n'.format(df_submission.shape))
print('TYPES:\n\n{}\n'.format(df_submission.dtypes))

df_submission.head()


ID match with test set: True

DF shape: (23490, 2)

TYPES:

employee_id    int64
is_promoted    int64
dtype: object



,employee_id,is_promoted
0,8724,0
1,74430,0
2,72255,0
3,38562,0
4,64486,0


In [351]:
# check employee uniqueness

print('Train data, all unique employee ID: {}'.format(len(df_train.employee_id.unique()) == df_train.shape[0]))
print('Test data, all unique employee ID: {}'.format(len(df_test.employee_id.unique()) == df_test.shape[0]))


Train data, all unique employee ID: True
Test data, all unique employee ID: True


## Data preprocessing  

- Missing values imputation 
- Categorical vars to dummy
- Train data split for training and validation subsets
- Target balancing  


In [583]:
# select columns for X and y
col = df_train.columns[(df_train.columns != 'is_promoted') & (df_train.columns != 'employee_id')]
X = df_train[col]
y = df_train['is_promoted']
X_test = df_test[col]


### Missing values imputation

In [584]:
# Education classes 
pd.crosstab(index=df_train.education, columns=['education'], normalize=True).round(2)


col_0,education
education,
Bachelor's,0.70
Below Secondary,0.02
Master's & above,0.28


In [585]:
# previous_year_rating classes 
pd.crosstab(index=df_train.previous_year_rating, columns=['previous_year_rating'], normalize=True).round(2)


col_0,previous_year_rating
previous_year_rating,
1.0,0.12
2.0,0.08
3.0,0.37
4.0,0.19
5.0,0.23


In [586]:
# combine training and test data for complete categories sets
combind = pd.concat([X, X_test], ignore_index=True)
print('Shape before: {}\n'.format([X.shape, X_test.shape, combind.shape]))

# Make a class 'Unknown' for missing values
combind['education'][combind.education.isnull()] = 'Unknown'
combind['previous_year_rating'][combind.previous_year_rating.isnull()] = 3.0

print('MISSING VALUES:\n\n{}\n\n'.format(combind.isnull().sum()))

# to dummies
combind = pd.get_dummies(combind, drop_first=True)
X = combind.iloc[:X.shape[0],]
X_test = combind.iloc[X.shape[0]:,]

print('Shape after: {}\n'.format([X.shape, X_test.shape]))


Shape before: [(54808, 12), (23490, 12), (78298, 12)]

MISSING VALUES:

department              0
region                  0
education               0
gender                  0
recruitment_channel     0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
dtype: int64


Shape after: [(54808, 54), (23490, 54)]



/Users/alekseyshipitsyn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/alekseyshipitsyn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### Scale

In [587]:
# scale
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X_test = scaler.transform(X_test)

### Train data split for training and validation subsets

In [606]:
# Split training data to training and validation subsets

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape


((41106, 54), (41106,), (13702, 54), (13702,))

## Modeling

In [607]:
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

f1_scorer = make_scorer(f1_score)


In [621]:
GridSearchCV.fit?

### Logistic regression

In [599]:
params = {'C': np.arange(1, 10)}

grid = GridSearchCV(LogisticRegression(), param_grid=params, scoring=f1_scorer, cv=3, n_jobs=8)

grid.fit(X_train, y_train)

grid.best_params_


/Users/alekseyshipitsyn/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


{'C': 9}

In [600]:
# Evaluation
model = grid.best_estimator_

f1_train = f1_score(y_true=y_train, y_pred=model.predict(X_train))
print('Score for training set: {}'.format(f1_train.round(3)))

f1_validation = f1_score(y_true=y_valid, y_pred=model.predict(X_valid))
print('Score for validation set: {}'.format(f1_validation.round(3)))


Score for training set: 0.411
Score for validation set: 0.407


### XGBoost

In [630]:
X_train = X
y_train = y

model = XGBClassifier(objective='binary:logistic', booster='gbtree', max_depth=4, 
                      n_estimators=200, eval_metric='auc')

params = {'learning_rate': [0.13, 0.14, 0.15, 0.16, 0.17],
          'scale_pos_weight': [2.3, 2.4, 2.5, 2.6, 2.7]}

grid = GridSearchCV(model, param_grid=params, scoring=f1_scorer, cv=5, n_jobs=8)

grid.fit(X_train, y_train)

grid.best_params_


{'learning_rate': 0.16, 'scale_pos_weight': 2.6}

In [612]:
# Evaluation
model = grid.best_estimator_

f1_train = f1_score(y_true=y_train, y_pred=model.predict(X_train))
print('Score for training set: {}'.format(f1_train.round(3)))

f1_validation = f1_score(y_true=y_valid, y_pred=model.predict(X_valid))
print('Score for validation set: {}'.format(f1_validation.round(3)))

#Score for training set: 0.6
#Score for validation set: 0.524

Score for training set: 0.6
Score for validation set: 0.524


In [631]:
# retrain the model on all training data

X_train = X
y_train = y

model = grid.best_estimator_

model.fit(X_train, y_train)

f1_train = f1_score(y_true=y_train, y_pred=model.predict(X_train))
print('Score for training data: {}'.format(f1_train.round(3)))

#Score for training data: 0.567


Score for training data: 0.572


## Submission

In [632]:
pred = model.predict(X_test)

df_submission['is_promoted'] = pred
print(df_submission.head())

df_submission.to_csv(folder + 'submission_14.csv', index=False)



   employee_id  is_promoted
0         8724            0
1        74430            0
2        72255            0
3        38562            0
4        64486            0


In [633]:
# rank at submission
rank = 110
participants = 6834
print('Acheaved rank is {} from {} participants, or top {} %'.format(
            rank, participants, round(100*rank/participants, 2)))

Acheaved rank is 110 from 6834 participants, or top 1.61 %
